In [1]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output, callback
from datetime import date, timedelta, datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay, BDay
import holidays
from workalendar.europe import UnitedKingdom 
import base64

def avg(listt):
    return sum(listt) / len(listt)

app = Dash(__name__)

image_filename = 'TDlogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')
image_data_uri = f"data:image/png;base64,{encoded_image}"

# Add the path to your assets folder
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.Img(src=image_data_uri, 
                         style={'height': '30px', 'marginRight': '10px'}),
                html.H1(
                    children="Metals Pricer Tool",
                    style={"fontSize": "30px", 'display': 'inline-block', 'verticalAlign': 'middle'},
                    className="header"
                )
            ],
            className="header",
            style={'display': 'flex', 'alignItems': 'center'}
        ),
        html.Div(
            children=[
                html.Div(children="Spot:", className="spot-title"),
                dcc.Input(id="spot", type="text", placeholder="", style={'marginRight': '10px'}),
                html.Div(id="output-spot")
            ],
            style={'marginTop': '20px'}
        ),
        html.Div(
            children=[
                html.Div(children="Forward Rate:", className="forward-title"),
                dcc.Input(id="forward-rate", type="text", placeholder="", style={'marginRight': '10px'}),
                html.Div(id="output-forward-rate")
            ],
            style={'marginTop': '20px'}
        ),
        html.Div(
            style={'display': 'flex', 'alignItems': 'center', 'marginTop': '20px'},
            children=[
                html.Div(
                    children=[
                        html.Div(children="Date Range:", className="menu-title"),
                        dcc.DatePickerRange(
                            id="my-date-picker-range",
                            min_date_allowed=date(1995, 1, 1),
                            max_date_allowed=date(2050, 12, 25),
                            initial_visible_month=date(2024, 8, 1),
                            end_date=date(2024, 8, 2)
                        ),
                    ],
                    style={'marginRight': '20px'}
                ),
                html.Div(
                    children=[
                        html.Div(children="Value Date:", className="menu-title"),
                        dcc.DatePickerSingle(
                            id="value-date",
                            min_date_allowed=date(1995, 1, 1),
                            max_date_allowed=date(2050, 12, 25),
                            initial_visible_month=date(2024, 8, 1),
                            date=None
                        ),
                    ]
                ),
            ]
        ),
        html.Div(id='output-container-date-picker-range', style={'marginTop': '20px'}),
        html.Div(id='output-spot-adjusted', style={'marginTop': '10px'}),
        html.Div(id='output-forward-rate-adjusted', style={'marginTop': '10px'}),
        html.Div(id='output-start-date-adjusted', style={'marginTop': '10px'}),
        html.Div(id='output-end-date-adjusted', style={'marginTop': '10px'}),
        html.Div(id='output-value-date-adjusted', style={'marginTop': '10px'}),
        html.Div(id='output-bid', style={'marginTop': '20px'})
    ]
)

@app.callback(
    [
        Output("output-bid", "children"),
        Output("output-spot-adjusted", "children"),
        Output("output-forward-rate-adjusted", "children"),
        Output("output-start-date-adjusted", "children"),
        Output("output-end-date-adjusted", "children"),
        Output("output-value-date-adjusted", "children")
    ],
    [
        Input("spot", "value"),
        Input("forward-rate", "value"),
        Input("my-date-picker-range", "start_date"),
        Input("my-date-picker-range", "end_date"),
        Input("value-date", "date")
    ]
)
def calculate_bid(spot, frd, start_date, end_date, value_date):
    output = []
    day_gaps = []
    dates = []
    delta = timedelta(days=1)
    cal = UnitedKingdom()

    adjusted_spot = f""
    adjusted_forward_rate = f""
    adjusted_start_date = f""
    adjusted_end_date = f""
    adjusted_value_date = f""

    if start_date is not None:
        start_date_object = date.fromisoformat(start_date)
        while not cal.is_working_day(start_date_object):
            start_date_object += timedelta(days=1)
        shifted_start_object = start_date_object
        for _ in range(2):  # Add 2 business days
            shifted_start_object += timedelta(days=1)
            while not cal.is_working_day(shifted_start_object):
                shifted_start_object += timedelta(days=1)
        adjusted_start_date = f"Shifted Start Date: {shifted_start_object}"
        output.append(html.P(f"Shifted Date: {shifted_start_object}"))

    if end_date is not None:
        end_date_object = date.fromisoformat(end_date)
        while not cal.is_working_day(end_date_object):
            end_date_object -= timedelta(days=1)
        adjusted_end_date = f"Adjusted End Date: {end_date_object}"

    if spot and frd and end_date is not None and value_date is not None:
        try:
            spot = float(spot)
            forward_rate = float(frd) / 100
            value_date_object = date.fromisoformat(value_date)
            adjusted_spot = f"Spot: {spot}"
            adjusted_forward_rate = f"Forward Rate: {frd}%"
            adjusted_value_date = f"Value Date: {value_date_object}"

            while shifted_start_object <= end_date_object:
                if cal.is_working_day(shifted_start_object):
                    dates.append(shifted_start_object.isoformat())
                    day_gaps.append((value_date_object - shifted_start_object).days)
                shifted_start_object += delta

            if len(day_gaps) > 0:
                average = avg(day_gaps)
                bid = spot * forward_rate * (average / 360)
                return (
                    f"Spread: ${bid:.2f}",
                    adjusted_spot,
                    adjusted_forward_rate,
                    adjusted_start_date,
                    adjusted_end_date,
                    adjusted_value_date
                )
            else:
                return (
                    "End date must be after start date or manual value date.",
                    adjusted_spot,
                    adjusted_forward_rate,
                    adjusted_start_date,
                    adjusted_end_date,
                    adjusted_value_date
                )

        except ValueError:
            return (
                "Invalid input. Please enter numeric values for Spot and Forward Rate.",
                adjusted_spot,
                adjusted_forward_rate,
                adjusted_start_date,
                adjusted_end_date,
                adjusted_value_date
            )

    return (
        "",
        adjusted_spot,
        adjusted_forward_rate,
        adjusted_start_date,
        adjusted_end_date,
        adjusted_value_date
    )

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
